In [1]:
import findspark
import folium

findspark.init()

from pyspark import sql, SparkConf, SparkContext

coordinates = (20.593684, 78.96288)
conf = SparkConf().setAppName("Read_CSV")
sc = SparkContext(conf=conf)
sqlContext = sql.SQLContext(sc)
map = folium.Map(location=coordinates, zoom_start=5)

dfSteden = sqlContext.read.csv("Steden.csv", header=True)
dfLucht = sqlContext.read.csv("Luchtvervuiling.csv", header=True)

In [ ]:
locations = dfSteden.select("lat", "lng", "city")
index = 0
c = '#3186cc'
new = dfLucht.join(locations, dfLucht.location == locations.city, how='left')
new2 = new.filter("lat != ''")
joinDF = new2.filter("lng != ''").collect()

for line in joinDF:
    if(joinDF[index][6] == 'NA' or not joinDF[index][6] is None  or (float(joinDF[index][6]) > 4)):
        c = '#ff0000'
    else:
        c = '#3186cc'
    folium.CircleMarker(
        location=[joinDF[index][13], joinDF[index][14]],
        radius=4,
        popup=joinDF[index][15],
        color=c,
        fill=True,
        fill_color='#3186cc'
    ).add_to(map)
    index = index + 1
    
display(map)